In [19]:
import torch

import src.dnn.datasets
import src.dnn.models
import src.dnn.test
import src.dnn.train
import src.dnn.utils

train_dataset, test_dataset = src.dnn.datasets.DATASETS['MNIST'](train_batch_size=128, test_batch_size=128, path='datasets')
#model = src.dnn.train.train(src.dnn.models.MODELS['LeNet5'](), cuda=True, train_dataset=train_dataset, optimizer_class=torch.optim.Adam, optimizer_args={}, loss=torch.nn.CrossEntropyLoss(), n_epochs=1)
model = src.dnn.utils.init_model(src.dnn.models.MODELS['LeNet5'], model_args={}, use_saved_model=True, model_save_path='models/lenet5.pkl', cuda=True, train_dataset=train_dataset, optimizer_class=torch.optim.Adam, optimizer_args={}, loss=torch.nn.CrossEntropyLoss(), n_epochs=1)
loaded_model = src.dnn.utils.load_model(src.dnn.models.MODELS['LeNet5'](), 'models/lenet5.pkl')
print('Done')

Done


In [20]:
print(dict(model['model'].named_parameters())['fc.f7.bias'])
print(dict(loaded_model.named_parameters())['fc.f7.bias'])
print(src.dnn.test.test(model['model'], test_dataset, loss=torch.nn.CrossEntropyLoss()))
print(src.dnn.test.test(loaded_model, test_dataset, loss=torch.nn.CrossEntropyLoss()))

Parameter containing:
tensor([ 0.0557,  0.0186, -0.0026, -0.1097,  0.0753, -0.0255,  0.0785,  0.0463,
         0.0458,  0.0891], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0557,  0.0186, -0.0026, -0.1097,  0.0753, -0.0255,  0.0785,  0.0463,
         0.0458,  0.0891], device='cuda:0', requires_grad=True)
{'accuracy': 0.9712, 'loss': 0.0007349000092595815}
{'accuracy': 0.9712, 'loss': 0.0007349000092595815}


In [15]:
torch.save(model['model'].state_dict(), 'models/lenet5.pkl')

In [39]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os

class LightningMNISTClassifier(pl.LightningModule):

    def __init__(self):
        super(LightningMNISTClassifier, self).__init__()

        self.enable = False

        # mnist images are (1, 28, 28) (channels, width, height) 
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)

        # layer 1 (b, 1*28*28) -> (b, 128)
        x = self.layer_1(x)
        x = torch.relu(x)

        # layer 2 (b, 128) -> (b, 256)
        x = self.layer_2(x)
        x = torch.relu(x)

        # layer 3 (b, 256) -> (b, 10)
        x = self.layer_3(x)

        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        print(loss)
        return {'test_loss': loss}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        # called at the end of the validation epoch
        # outputs is an array with what you returned in validation_step for each batch
        # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}] 
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def prepare_data(self):
        # transforms for images
        transform=transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.1307,), (0.3081,))])
        
        # prepare transforms standard to MNIST
        mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
        
        self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
        self.mnist_test = mnist_test

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=64)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=64)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def on_test_start(self):
        if self.enable:
            self.layer_1 = torch.nn.Sequential(self.layer_1, torch.nn.Softmax())

    def enable_(self):
        self.enable = True

# train
model = LightningMNISTClassifier()
trainer = pl.Trainer(gpus=1, max_epochs=1)

trainer.fit(model)

print(trainer.test(model))

#model.layer_1 = torch.nn.Sequential(model.layer_1, torch.nn.Softmax())
model.enable_()
print(trainer.test(model))

# trainer.test() does not work if we change the model
#for i, test_batch in enumerate(model.test_dataloader()):
#    x, y = test_batch
#    logits = model.forward(x.to(model.device))
#    loss = model.cross_entropy_loss(logits, y.to(model.device))
#    print({'test_loss': loss})

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   
Epoch 1: 100%|██████████| 939/939 [00:12<00:00, 74.61it/s, loss=0.130, v_num=14]
Testing: 0it [00:00, ?it/s]tensor(0.0636, device='cuda:0')
tensor(0.0369, device='cuda:0')
tensor(0.0620, device='cuda:0')
tensor(0.1626, device='cuda:0')
tensor(0.0678, device='cuda:0')
tensor(0.1659, device='cuda:0')
tensor(0.0874, device='cuda:0')
tensor(0.1190, device='cuda:0')
Testing:   5%|▌         | 8/157 [00:00<00:02, 73.88it/s]tensor(0.0809, device='cuda:0')
tensor(0.2459, device='cuda:0')
tensor(0.1248, device='cuda:0')
tensor(0.1696, device='cuda:0')
tensor(0.0275, device='cuda:0')
tensor(0.0664, device='cuda:0')
tensor(0.2482, device='cuda:0')
tensor(0.1628, device='cuda:0')
tensor(0.1335, device='cuda:0')
Testing:  11%|█         | 17/157 [00